In [54]:
import math
from itertools import combinations, permutations

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined 24-bit color palette (RGB tuples)
    palette = [
        ('red', (255, 0, 0)), ('blue', (0, 0, 255)), ('green', (0, 255, 0)),
        ('yellow', (255, 255, 0)), ('cyan', (0, 255, 255)), ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)), ('purple', (128, 0, 128)), ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)), ('teal', (0, 128, 128)), ('violet', (238, 130, 238)),
        ('pink', (255, 192, 203)), ('olive', (128, 128, 0)), ('lime', (0, 255, 0))
    ]
    color_names = [c[0] for c in palette]  # Extract color names only

    # Calculate block capacities
    B_color = math.comb(2**24, n)  # Theoretical (16.7M for n=1)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm))

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(f'\\textcolor{{{color}}}{{{cover_chars[pos]}}}')
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Inputs from the problem
secret_message = "underlying physiological mechanisms"
cover_text = "Only boats catch connotes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels"
n = 10
pi = list(range(20))  # Permutation key

# Generate stego-text
stego_output = embed_k_block(secret_message, cover_text, n, pi)

# Print first 181 characters (as in the example)
print(stego_output[:len(stego_output)])

\textcolor{olive}{O}\textcolor{magenta}{n}\textcolor{orange}{l}\textcolor{red}{y}\textcolor{purple}{ }\textcolor{green}{b}\textcolor{cyan}{o}\textcolor{gray}{a}\textcolor{yellow}{t}\textcolor{lime}{s}\textcolor{brown}{ }\textcolor{red}{c}\textcolor{yellow}{a}\textcolor{cyan}{t}\textcolor{blue}{c}\textcolor{green}{h}\textcolor{magenta}{ }\textcolor{teal}{c}\textcolor{pink}{o}\textcolor{lime}{n}notes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels


In [55]:
import math
from itertools import combinations, permutations

class ConsoleColor:
    """ANSI color codes for console output"""
    COLORS = {
        'red': '\033[91m',
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'cyan': '\033[96m',
        'magenta': '\033[95m',
        'orange': '\033[38;5;208m',
        'purple': '\033[38;5;129m',
        'brown': '\033[38;5;130m',
        'gray': '\033[38;5;240m',
        'teal': '\033[38;5;30m',
        'violet': '\033[38;5;177m',
        'pink': '\033[38;5;211m',
        'olive': '\033[38;5;100m',
        'lime': '\033[38;5;154m',
        'reset': '\033[0m'
    }

    @staticmethod
    def color_char(char, color_name):
        """Color a single character for console output"""
        return f"{ConsoleColor.COLORS.get(color_name, '')}{char}{ConsoleColor.COLORS['reset']}"

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette
    palette = [
        'red', 'blue', 'green', 'yellow', 'cyan', 'magenta',
        'orange', 'purple', 'brown', 'gray', 'teal', 'violet',
        'pink', 'olive', 'lime'
    ]

    # Calculate block capacities
    B_color = math.comb(len(palette), n)  # Using actual palette size instead of 2^24
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0
    #print(k)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0') if k > 0 else ''

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, palette)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(ConsoleColor.color_char(cover_chars[pos], color))
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Input parameters
secret_message = "underlying physiological mechanisms"
cover_text = "Only boats catch connotes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels"
n = 10
pi = list(range(20))  # Permutation key

# Generate and print colored text
colored_output = embed_k_block(secret_message, cover_text, n, pi)
secret = secret_message
# Print the first 181 characters with actual colors in console
print(colored_output[:len(colored_output)])

Only boats catch connotes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels


In [32]:
import math
from itertools import combinations, permutations

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined 24-bit color palette (RGB tuples)
    palette = [
        ('red', (255, 0, 0)), ('blue', (0, 0, 255)), ('green', (0, 255, 0)),
        ('yellow', (255, 255, 0)), ('cyan', (0, 255, 255)), ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)), ('purple', (128, 0, 128)), ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)), ('teal', (0, 128, 128)), ('violet', (238, 130, 238)),
        ('pink', (255, 192, 203)), ('olive', (128, 128, 0)), ('lime', (0, 255, 0))
    ]
    color_names = [c[0] for c in palette]  # Extract color names only

    # Calculate block capacities
    B_color = math.comb(2**24, n)  # Theoretical (16.7M for n=1)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm))

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(f'\\textcolor{{{color}}}{{{cover_chars[pos]}}}')
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Inputs from the problem
secret_message = "Success starts with small steps. Every morning brings new opportunities. The world is full of possibilities dare to explore! Challenges make us stronger; failures teach resilience. Smile often, kindness costs nothing. Time flies, so chase your dreams today. Learn, grow, and never stop believing. A positive mind creates a brighter future. What is your next goal? Take action now! Life is too short for regrets. Shine bright, stay curious and inspire others. Remember: every effort counts. Keep going! Progress happens when you push forward"
cover_text = "Only boats catch connotes of the islands sober wines only ships wrap the slips on the clats of twining lines only flags flap in tags with color that assigns only passage on vessels"
n = 10
pi = list(range(20))  # Permutation key

# Generate stego-text
stego_output = embed_k_block(secret_message, cover_text, n, pi)

# Print first 181 characters (as in the example)
print(stego_output[:len(stego_output)])

\textcolor{red}{O}\textcolor{violet}{n}\textcolor{orange}{l}\textcolor{gray}{y}\textcolor{magenta}{ }\textcolor{blue}{b}\textcolor{yellow}{o}\textcolor{cyan}{a}\textcolor{brown}{t}\textcolor{pink}{s}\textcolor{teal}{ }\textcolor{yellow}{c}\textcolor{cyan}{a}\textcolor{olive}{t}\textcolor{green}{c}\textcolor{lime}{h}\textcolor{brown}{ }\textcolor{magenta}{c}\textcolor{red}{o}\textcolor{orange}{n}\textcolor{lime}{n}\textcolor{purple}{o}\textcolor{brown}{t}\textcolor{pink}{e}\textcolor{red}{s}\textcolor{magenta}{ }\textcolor{orange}{o}\textcolor{yellow}{f}\textcolor{gray}{ }\textcolor{cyan}{t}\textcolor{magenta}{h}\textcolor{teal}{e}\textcolor{olive}{ }\textcolor{yellow}{i}\textcolor{gray}{s}\textcolor{green}{l}\textcolor{orange}{a}\textcolor{cyan}{n}\textcolor{purple}{d}\textcolor{violet}{s}\textcolor{cyan}{ }\textcolor{magenta}{s}\textcolor{orange}{o}\textcolor{violet}{b}\textcolor{gray}{e}\textcolor{brown}{r}\textcolor{blue}{ }\textcolor{teal}{w}\textcolor{purple}{i}\textcolor{red}{n}\

In [56]:
import math
from itertools import combinations, permutations

class ConsoleColor:
    """ANSI color codes for console output"""
    COLORS = {
        'red': '\033[91m',
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'cyan': '\033[96m',
        'magenta': '\033[95m',
        'orange': '\033[38;5;208m',
        'purple': '\033[38;5;129m',
        'brown': '\033[38;5;130m',
        'gray': '\033[38;5;240m',
        'teal': '\033[38;5;30m',
        'violet': '\033[38;5;177m',
        'pink': '\033[38;5;211m',
        'olive': '\033[38;5;100m',
        'lime': '\033[38;5;154m',
        'reset': '\033[0m'
    }

    @staticmethod
    def color_char(char, color_name):
        """Color a single character for console output"""
        return f"{ConsoleColor.COLORS.get(color_name, '')}{char}{ConsoleColor.COLORS['reset']}"

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette
    palette = [
        'red', 'blue', 'green', 'yellow', 'cyan', 'magenta',
        'orange', 'purple', 'brown', 'gray', 'teal', 'violet',
        'pink', 'olive', 'lime'
    ]

    # Calculate block capacities
    B_color = math.comb(len(palette), n)  # Using actual palette size instead of 2^24
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0
    #print(k)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0') if k > 0 else ''

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, palette)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(ConsoleColor.color_char(cover_chars[pos], color))
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Input parameters
secret_message = "The sun dipped below the horizon, painting the sky in shades of orange and purple. A cool breeze rustled the leaves, carrying the scent of fresh rain. Distant laughter echoed from a nearby park as people enjoyed the evening. The city lights flickered to life, casting long shadows on the pavement. A lone jogger passed by, their footsteps rhythmic against the quiet hum of traffic. Somewhere, a dog barked, and a child’s voice called out in excitement. The world felt alive, pulsing with energy even as night fell. Moments like these made everything seem peaceful, fleeting yet perfect."
cover_text = "Only boats catch connotes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels"
n = 10
pi = list(range(20))  # Permutation key

# Generate and print colored text
colored_output = embed_k_block(secret_message, cover_text, n, pi)
secret = secret_message
# Print the first 181 characters with actual colors in console
print(colored_output[:len(colored_output)])

Only boats catch connotes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels


In [57]:
import math
from itertools import combinations, permutations

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined 24-bit color palette (RGB tuples)
    palette = [
        ('red', (255, 0, 0)), ('blue', (0, 0, 255)), ('green', (0, 255, 0)),
        ('yellow', (255, 255, 0)), ('cyan', (0, 255, 255)), ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)), ('purple', (128, 0, 128)), ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)), ('teal', (0, 128, 128)), ('violet', (238, 130, 238)),
        ('pink', (255, 192, 203)), ('olive', (128, 128, 0)), ('lime', (0, 255, 0))
    ]
    color_names = [c[0] for c in palette]  # Extract color names only

    # Calculate block capacities
    B_color = math.comb(2**24, n)  # Theoretical (16.7M for n=1)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm))

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(f'\\textcolor{{{color}}}{{{cover_chars[pos]}}}')
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Inputs from the problem
secret_message = "behind using a cover text is to hide the presence  of secret messages the presence of embedded messages in the resulting stego-text cannot be easily discovered by anyone except the intended recipient."
cover_text = "in the research area of text steganography, algorithms based on font format have advantages of great capacity, good imperceptibility and wide application range. However, little work on steganalysis for such algorithms has been reported in the literature. Based on the fact that the statistic features of font format will be changed after using font-format-based steganographic algorithms, we present a novel support vector machine-based steganalysis algorithm to detect whether hidden information exists or not. This algorithm can not only effectively detect the existence of hidden information, but also estimate the hidden information length according to variations of font attribute value. As shown by experimental results, the detection accuracy of our algorithm reaches as high as 99.3 \% when the hidden information length is at least 16 bits."
n = 10
pi = list(range(20))  # Permutation key

# Generate stego-text
stego_output = embed_k_block(secret_message, cover_text, n, pi)

# Print first 181 characters (as in the example)
print(stego_output[:len(stego_output)])

\textcolor{magenta}{i}\textcolor{teal}{n}\textcolor{olive}{ }\textcolor{yellow}{t}\textcolor{green}{h}\textcolor{violet}{e}\textcolor{purple}{ }\textcolor{pink}{r}\textcolor{blue}{e}\textcolor{gray}{s}\textcolor{orange}{e}\textcolor{teal}{a}\textcolor{lime}{r}\textcolor{gray}{c}\textcolor{violet}{h}\textcolor{pink}{ }\textcolor{magenta}{a}\textcolor{green}{r}\textcolor{purple}{e}\textcolor{yellow}{a}\textcolor{gray}{ }\textcolor{teal}{o}\textcolor{violet}{f}\textcolor{cyan}{ }\textcolor{purple}{t}\textcolor{red}{e}\textcolor{blue}{x}\textcolor{olive}{t}\textcolor{green}{ }\textcolor{pink}{s}\textcolor{blue}{t}\textcolor{purple}{e}\textcolor{teal}{g}\textcolor{red}{a}\textcolor{green}{n}\textcolor{lime}{o}\textcolor{olive}{g}\textcolor{orange}{r}\textcolor{cyan}{a}\textcolor{magenta}{p}\textcolor{teal}{h}\textcolor{pink}{y}\textcolor{gray}{,}\textcolor{cyan}{ }\textcolor{purple}{a}\textcolor{red}{l}\textcolor{blue}{g}\textcolor{olive}{o}\textcolor{lime}{r}\textcolor{brown}{i}\textcolor{

In [58]:
import math
from itertools import combinations, permutations

class ConsoleColor:
    """ANSI color codes for console output"""
    COLORS = {
        'red': '\033[91m',
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'cyan': '\033[96m',
        'magenta': '\033[95m',
        'orange': '\033[38;5;208m',
        'purple': '\033[38;5;129m',
        'brown': '\033[38;5;130m',
        'gray': '\033[38;5;240m',
        'teal': '\033[38;5;30m',
        'violet': '\033[38;5;177m',
        'pink': '\033[38;5;211m',
        'olive': '\033[38;5;100m',
        'lime': '\033[38;5;154m',
        'reset': '\033[0m'
    }

    @staticmethod
    def color_char(char, color_name):
        """Color a single character for console output"""
        return f"{ConsoleColor.COLORS.get(color_name, '')}{char}{ConsoleColor.COLORS['reset']}"

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette
    palette = [
        'red', 'blue', 'green', 'yellow', 'cyan', 'magenta',
        'orange', 'purple', 'brown', 'gray', 'teal', 'violet',
        'pink', 'olive', 'lime'
    ]

    # Calculate block capacities
    B_color = math.comb(len(palette), n)  # Using actual palette size instead of 2^24
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0
    #print(k)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0') if k > 0 else ''

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, palette)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(ConsoleColor.color_char(cover_chars[pos], color))
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Input parameters
secret_message = "behind using a cover text is to hide the presence  of secret messages the presence of embedded messages in the resulting stego-text cannot be easily discovered by anyone except the intended recipient."
cover_text = "in the research area of text steganography, algorithms based on font format have advantages of great capacity, good imperceptibility and wide application range. However, little work on steganalysis for such algorithms has been reported in the literature. Based on the fact that the statistic features of font format will be changed after using font-format-based steganographic algorithms, we present a novel support vector machine-based steganalysis algorithm to detect whether hidden information exists or not. This algorithm can not only effectively detect the existence of hidden information, but also estimate the hidden information length according to variations of font attribute value. As shown by experimental results, the detection accuracy of our algorithm reaches as high as 99.3 \% when the hidden information length is at least 16 bits."
n = 10
pi = list(range(20))  # Permutation key

# Generate and print colored text
colored_output = embed_k_block(secret_message, cover_text, n, pi)
secret = secret_message
# Print the first 181 characters with actual colors in console
print(colored_output[:len(colored_output)])

in the research area of text steganography, algorithms based on font format have advantages of great capacity, good imperceptibility and wide application range. However, little work on steganalysis for such algorithms has been reported in the literature. Based on the fact that the statistic features of font format will be changed after using font-format-based steganographic algorithms, we present a novel support vector machine-based steganalysis algorithm to detect whether hidden information exists or not. This algorithm can not only effectively detect the existence of hidden information, but also estimate the hidden information length according to variations of font attribute value. As shown by experimental results, the detection accuracy of our algorithm reaches as high as 99.3 \% when the hidden information length is at least 16 bits.


In [52]:
import math
from itertools import combinations, permutations

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined 24-bit color palette (RGB tuples)
    palette = [
        ('red', (255, 0, 0)), ('blue', (0, 0, 255)), ('green', (0, 255, 0)),
        ('yellow', (255, 255, 0)), ('cyan', (0, 255, 255)), ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)), ('purple', (128, 0, 128)), ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)), ('teal', (0, 128, 128)), ('violet', (238, 130, 238)),
        ('pink', (255, 192, 203)), ('olive', (128, 128, 0)), ('lime', (0, 255, 0))
    ]
    color_names = [c[0] for c in palette]  # Extract color names only

    # Calculate block capacities
    B_color = math.comb(2**24, n)  # Theoretical (16.7M for n=1)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm))

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(f'\\textcolor{{{color}}}{{{cover_chars[pos]}}}')
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Inputs from the problem
secret_message = "The field of digital information security continues to evolve rapidly as technological advancements present both new opportunities and challenges. Among various data protection methods, text steganography has emerged as a particularly interesting approach due to its ability to conceal information within seemingly innocuous textual content. This technique differs fundamentally from cryptography, as it focuses on hiding the very existence of secret messages rather than merely scrambling their content. Recent developments in font-based steganography have demonstrated remarkable improvements in capacity and undetectability. These methods leverage subtle modifications to font characteristics such as kerning, glyph shapes, or Unicode variations to embed information without visible alterations to the text. The advantages of such approaches include compatibility with standard word processors, platform independence, and resistance to simple visual inspection. However, the effectiveness of any steganographic system ultimately depends on its ability to withstand steganalysis - the art of detecting hidden information. Current research indicates that machine learning techniques, particularly support vector machines and deep neural networks, show great promise in identifying font-based steganography. These detection methods analyze statistical properties of text formatting that may reveal the presence of hidden data, even when such traces are imperceptible to human observers. The ongoing arms race between steganography and steganalysis drives continuous innovation in both fields. As embedding techniques become more sophisticated, detection methods must correspondingly advance to maintain security standards. This dynamic creates exciting research opportunities in digital forensics, particularly in developing universal detectors capable of identifying multiple steganographic techniques across different text formats. Future directions in text steganography research may explore adaptive algorithms that can modify their embedding patterns based on the surrounding text content, making detection even more challenging. Additionally, the integration of natural language processing techniques could enable more intelligent information hiding that accounts for semantic context rather than just syntactic features. From an application perspective, font-based steganography finds use in various domains including secure communications, copyright protection, and anti-counterfeiting measures. Its relatively low computational requirements."
cover_text = "in the research area of text steganography, algorithms based on font format have advantages of great capacity, good imperceptibility and wide application range. However, little work on steganalysis for such algorithms has been reported in the literature. Based on the fact that the statistic features of font format will be changed after using font-format-based steganographic algorithms, we present a novel support vector machine-based steganalysis algorithm to detect whether hidden information exists or not. This algorithm can not only effectively detect the existence of hidden information, but also estimate the hidden information length according to variations of font attribute values. As shown by experimental results, the detection accuracy of our algorithm reaches as high as 99.3 \%, when the hidden information length is at least 16 bits."
n = 10
pi = list(range(20))  # Permutation key

# Generate stego-text
stego_output = embed_k_block(secret_message, cover_text, n, pi)

# Print first 181 characters (as in the example)
print(stego_output[:len(stego_output)])

\textcolor{blue}{i}\textcolor{magenta}{n}\textcolor{lime}{ }\textcolor{olive}{t}\textcolor{teal}{h}\textcolor{green}{e}\textcolor{gray}{ }\textcolor{yellow}{r}\textcolor{red}{e}\textcolor{purple}{s}\textcolor{blue}{e}\textcolor{brown}{a}\textcolor{magenta}{r}\textcolor{cyan}{c}\textcolor{green}{h}\textcolor{olive}{ }\textcolor{pink}{a}\textcolor{gray}{r}\textcolor{violet}{e}\textcolor{red}{a}\textcolor{teal}{ }\textcolor{olive}{o}\textcolor{yellow}{f}\textcolor{blue}{ }\textcolor{cyan}{t}\textcolor{brown}{e}\textcolor{violet}{x}\textcolor{green}{t}\textcolor{red}{ }\textcolor{lime}{s}\textcolor{blue}{t}\textcolor{yellow}{e}\textcolor{green}{g}\textcolor{purple}{a}\textcolor{lime}{n}\textcolor{red}{o}\textcolor{brown}{g}\textcolor{orange}{r}\textcolor{pink}{a}\textcolor{magenta}{p}\textcolor{purple}{h}\textcolor{violet}{y}\textcolor{teal}{,}\textcolor{orange}{ }\textcolor{brown}{a}\textcolor{green}{l}\textcolor{yellow}{g}\textcolor{lime}{o}\textcolor{olive}{r}\textcolor{blue}{i}\textcol

In [60]:
import math
from itertools import combinations, permutations

class ConsoleColor:
    """ANSI color codes for console output"""
    COLORS = {
        'red': '\033[91m',
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'cyan': '\033[96m',
        'magenta': '\033[95m',
        'orange': '\033[38;5;208m',
        'purple': '\033[38;5;129m',
        'brown': '\033[38;5;130m',
        'gray': '\033[38;5;240m',
        'teal': '\033[38;5;30m',
        'violet': '\033[38;5;177m',
        'pink': '\033[38;5;211m',
        'olive': '\033[38;5;100m',
        'lime': '\033[38;5;154m',
        'reset': '\033[0m'
    }

    @staticmethod
    def color_char(char, color_name):
        """Color a single character for console output"""
        return f"{ConsoleColor.COLORS.get(color_name, '')}{char}{ConsoleColor.COLORS['reset']}"

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette
    palette = [
        'red', 'blue', 'green', 'yellow', 'cyan', 'magenta',
        'orange', 'purple', 'brown', 'gray', 'teal', 'violet',
        'pink', 'olive', 'lime'
    ]

    # Calculate block capacities
    B_color = math.comb(len(palette), n)  # Using actual palette size instead of 2^24
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0
    #print(k)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0') if k > 0 else ''

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, palette)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(ConsoleColor.color_char(cover_chars[pos], color))
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Input parameters
secret_message = "The field of digital information security continues to evolve rapidly as technological advancements present both new opportunities and challenges. Among various data protection methods, text steganography has emerged as a particularly interesting approach due to its ability to conceal information within seemingly innocuous textual content. This technique differs fundamentally from cryptography, as it focuses on hiding the very existence of secret messages rather than merely scrambling their content. Recent developments in font-based steganography have demonstrated remarkable improvements in capacity and undetectability. These methods leverage subtle modifications to font characteristics such as kerning, glyph shapes, or Unicode variations to embed information without visible alterations to the text. The advantages of such approaches include compatibility with standard word processors, platform independence, and resistance to simple visual inspection. However, the effectiveness of any steganographic system ultimately depends on its ability to withstand steganalysis - the art of detecting hidden information. Current research indicates that machine learning techniques, particularly support vector machines and deep neural networks, show great promise in identifying font-based steganography. These detection methods analyze statistical properties of text formatting that may reveal the presence of hidden data, even when such traces are imperceptible to human observers. The ongoing arms race between steganography and steganalysis drives continuous innovation in both fields. As embedding techniques become more sophisticated, detection methods must correspondingly advance to maintain security standards. This dynamic creates exciting research opportunities in digital forensics, particularly in developing universal detectors capable of identifying multiple steganographic techniques across different text formats. Future directions in text steganography research may explore adaptive algorithms that can modify their embedding patterns based on the surrounding text content, making detection even more challenging. Additionally, the integration of natural language processing techniques could enable more intelligent information hiding that accounts for semantic context rather than just syntactic features. From an application perspective, font-based steganography finds use in various domains including secure communications, copyright protection, and anti-counterfeiting measures. Its relatively low computational requirements."
cover_text = "in the research area of text steganography, algorithms based on font format have advantages of great capacity, good imperceptibility and wide application range. However, little work on steganalysis for such algorithms has been reported in the literature. Based on the fact that the statistic features of font format will be changed after using font-format-based steganographic algorithms, we present a novel support vector machine-based steganalysis algorithm to detect whether hidden information exists or not. This algorithm can not only effectively detect the existence of hidden information, but also estimate the hidden information length according to variations of font attribute values. As shown by experimental results, the detection accuracy of our algorithm reaches as high as 99.3 \%, when the hidden information length is at least 16 bits."
n = 10
pi = list(range(20))  # Permutation key

# Generate and print colored text
colored_output = embed_k_block(secret_message, cover_text, n, pi)
secret = secret_message
# Print the first 181 characters with actual colors in console
print(colored_output[:len(colored_output)])

in the research area of text steganography, algorithms based on font format have advantages of great capacity, good imperceptibility and wide application range. However, little work on steganalysis for such algorithms has been reported in the literature. Based on the fact that the statistic features of font format will be changed after using font-format-based steganographic algorithms, we present a novel support vector machine-based steganalysis algorithm to detect whether hidden information exists or not. This algorithm can not only effectively detect the existence of hidden information, but also estimate the hidden information length according to variations of font attribute values. As shown by experimental results, the detection accuracy of our algorithm reaches as high as 99.3 \%, when the hidden information length is at least 16 bits.
